In [1]:
!pip uninstall gensim
!pip install gensim==3.8.3
!pip install nltk

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gensim-3.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gensim/*
Proceed (y/n)? Y
  Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
import pandas as pd
from datetime import datetime
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA


In [3]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = '/content/drive/My Drive/CS 48000 Project'

Mounted at /content/drive


In [4]:
df = pd.read_csv(DATA_PATH+'/SO_data_error_queries.csv')

In [5]:
dfa = pd.read_csv(DATA_PATH+'/SO_error_titles_answer_data.csv')

In [6]:
df.shape

(25068, 12)

In [7]:
df.dtypes

Unnamed: 0              int64
tags                   object
is_answered              bool
view_count              int64
answer_count            int64
score                   int64
last_activity_date     object
creation_date          object
question_id             int64
link                   object
title                  object
accepted_answer_id    float64
dtype: object

In [8]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

def preprocess(text: str):
    
    stop_words = set(stopwords.words('english'))
    stop_words.add("python")
    
    tokenizer = RegexpTokenizer(r'\w+')

    text = " ".join(tokenizer.tokenize(text))

    #print("w/o punctuation:", text)
    word_tokens = word_tokenize(text)
    
    #print("tokens:",  str(word_tokens))

    lower_case = [w.lower() for w in word_tokens]

    #print("lowercase: ",str(lower_case))
    filtered_sentence = [w for w in lower_case if not w in stop_words]
    #print("stop words:",str(filtered_sentence))
    ps = PorterStemmer()

    stemmed_text = [ps.stem(w) for w in filtered_sentence]
    #print("stemmed:",str(stemmed_text))

    return " ".join(stemmed_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
from sklearn.model_selection import train_test_split
train= df[['title','question_id']]
train['title'] = train['title'].apply(preprocess)

In [12]:
def read_corpus(fname, tokens_only=False):
        for i, line in enumerate(fname):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [13]:
train_corpus = pd.DataFrame()
train_corpus["title"] = list(read_corpus(train["title"]))
train_corpus["question_id"] = train["question_id"]

In [14]:
train_corpus[:2]

,title,question_id
0,"([error, instal, geopanda, gdal, api, version,...",54734667
1,"([unicod, error, unicodeescap, codec, decod, b...",37400974


In [15]:
model = gensim.models.doc2vec.Doc2Vec(epochs=20)

In [16]:
model.build_vocab(train_corpus["title"])

In [17]:
model.train(train_corpus["title"], total_examples=model.corpus_count, epochs=model.epochs)

In [18]:
with open(DATA_PATH+"/manuel_errors.txt","r") as f:
  p_list= [l.strip() for l in f.readlines()]

In [20]:
p_list[0:5]

['TypeError: Connect() with one argument',
 'ModuleNot FoundError virtual',
 'ValueError: NumPy bool array can’t assign 100 input to 90 output where mask true',
 'Pandas data frame unhashable type error',
 'Google-trans-new throws an error on previously working code in Python']

In [22]:
result = dict()
id = 0

for p in p_list:
  result[id] = dict()
  result[id]["Test Input"] = p
  p = preprocess(p)
  inf_vec = model.infer_vector(p.split())
  sims = model.docvecs.most_similar([inf_vec], topn=len(model.docvecs))
  result[id]["The Closest"] = dict()
  result[id]["The Closest"]["ID"] = int(sims[0][0])
  result[id]["The Closest"]["Words"] =  df["title"][df["question_id"] == train_corpus.iloc[sims[0][0]]["question_id"]].to_string()
  result[id]["The Closest"]["isValid"] =  True

  result[id]["Second Closest"] = dict()
  result[id]["Second Closest"]["ID"] = int(sims[1][0])
  result[id]["Second Closest"]["Words"] =  df["title"][df["question_id"] == train_corpus.iloc[sims[1][0]]["question_id"]].to_string()
  result[id]["Second Closest"]["isValid"] =  True

  id+=1

In [23]:
df["title"][df["question_id"] == train_corpus.iloc[sims[0][0]]["question_id"]]

6218    python pygame pygame error display surface quit
Name: title, dtype: object

In [24]:
with open ("eval.json","w") as file:
  json.dump(result, file,indent=4)

In [30]:
f = open(DATA_PATH+'/eval.json')
data = json.load(f)
list(data.items())[0:3]

[('0',
  {'Second Closest': {'ID': '21025    38616858',
    'Similarity': 0.5582849979400635,
    'Words': '21025    cqlsh connection error ref take keyword arguments',
    'isValid': True},
   'Test Input': 'TypeError: Connect() with one argument',
   'The Closest': {'ID': '6536    19143091',
    'Similarity': 0.797059178352356,
    'Words': '6536    typeerror connect takes exactly one argument',
    'isValid': True}}),
 ('1',
  {'Second Closest': {'ID': '9866    67312258',
    'Similarity': 0.9315162301063538,
    'Words': '9866    unboundlocalerror local variable var reference...',
    'isValid': False},
   'Test Input': 'ModuleNot FoundError virtual',
   'The Closest': {'ID': '21151    58577235',
    'Similarity': 0.9521731734275818,
    'Words': '21151    error activating virtual environment',
    'isValid': False}}),
 ('2',
  {'Second Closest': {'ID': '542    71880746',
    'Similarity': 0.7524867057800293,
    'Words': '542    xarray missingspatialdimensionerror assign coo...',


In [31]:
for id in data:
  p = data[id]["Test Input"]
  p = preprocess(p)
  inf_vec = model.infer_vector(p.split())
  sims = model.docvecs.most_similar([inf_vec], topn=len(model.docvecs))
  data[id]["The Closest"]["Similarity"] = float(sims[0][1])
  data[id]["Second Closest"]["Similarity"] = float(sims[1][1])
  data[id]["The Closest"]["Words"] =  df["title"][df["question_id"] == train_corpus.iloc[sims[0][0]]["question_id"]].to_string()
  data[id]["Second Closest"]["Words"] =  df["title"][df["question_id"] == train_corpus.iloc[sims[1][0]]["question_id"]].to_string()
  data[id]["Second Closest"]["ID"] =  df["question_id"][df["question_id"] == train_corpus.iloc[sims[1][0]]["question_id"]].to_string()
  data[id]["The Closest"]["ID"] =  df["question_id"][df["question_id"] == train_corpus.iloc[sims[0][0]]["question_id"]].to_string()

"When I use a dictionary to translate numbers written in Spanish words to numeric characters, I get a KeyError: 'y'. What should I do to solve?"

"Google-trans-new throws an error on previously working code in Python"

"Import error while using custom modules in Python 3"

"Although there is a replit error with the google API, it works in Pycharm"

"An AttributeErrorr occurs when SQLAlchemy is installed with Poetry"

"When combining two df in pandas, ValueError: Shape of passed values is (6, 6), indices indicate (4, 6) error"

"When using Flask, a PyCUDA context error happens"

In [111]:
question = input("Enter your question about python: ")

p = preprocess(question)
inf_vec = model.infer_vector(p.split())
sims = model.docvecs.most_similar([inf_vec], topn=len(model.docvecs))

question_id1 =  str(train_corpus.iloc[sims[0][0]]["question_id"])
similarity1 = float(sims[0][1])
t1=df["title"][df["question_id"] == train_corpus.iloc[sims[0][0]]["question_id"]].to_string()
title1 = t1[t1.find(" "):].strip()
question_id2 =  str(train_corpus.iloc[sims[1][0]]["question_id"])
similarity2 = float(sims[1][1])

t2=df["title"][df["question_id"] == train_corpus.iloc[sims[1][0]]["question_id"]].to_string()
title2 =  t2[t2.find(" "):].strip()

isValid1=False
isValid2=False

for item in data.items():
  if item[1]["The Closest"]["ID"].split()[1]==question_id1:
    isValid1=item[1]["The Closest"]["isValid"]
  if item[1]["Second Closest"]["ID"].split()[1]==question_id2:
    isValid2=item[1]["Second Closest"]["isValid"]

if isValid1:
    print("This question sounds similar to yours:",title1)
    answers=dfa[dfa["question_id"]==int(question_id1)]
    print("Links and explanations to answers to this question:")
    counter=1
    for index, row in answers.iterrows():
        print(str(counter)+".",row["link"] )
        soup = BeautifulSoup(row["body"])
        print(soup.get_text())
        counter+=1
    
elif isValid2:
    print("This question sounds similar to yours:",title2)
    answers=dfa[dfa["question_id"]==int(question_id2)]
    print("Links and explanations to answers to this question:")
    counter=1
    for index, row in answers.iterrows():
        print(str(counter)+".",row["link"] )
        soup = BeautifulSoup(row["body"])
        print(soup.get_text())
        counter+=1
else:
    print("We couldn't find any matches :(")

Enter your question about python: When using Flask, a PyCUDA context error happens
We couldn't find any matches :(


In [ ]:
from bs4 import BeautifulSoup

In [94]:
totalSimilarity = 0
countFirst = 0
countSecond = 0
countValid = 0

wrong_similarity= 0
for id in data:
  if data[id]["The Closest"]["isValid"] == True:
    countFirst+=1
    countValid+=1
    totalSimilarity += data[id]["The Closest"]["Similarity"]
  elif data[id]["Second Closest"]["isValid"] == True and data[id]["The Closest"]["isValid"] == False:
    countSecond+=1
    countValid+=1
    totalSimilarity += data[id]["Second Closest"]["Similarity"]
  elif  data[id]["Second Closest"]["isValid"] == False and data[id]["The Closest"]["isValid"] == False:
    wrong_similarity += data[id]["The Closest"]["Similarity"]

In [91]:
print("Percentage of times when the first suggestion is true:",countFirst/len(data))

Percentage of times when the first suggestion is true: 0.5396039603960396


In [92]:
print("Percentage of times when only the second suggestion is true:",countSecond/len(data))

Percentage of times when only the second suggestion is true: 0.0594059405940594


In [93]:
print("Percentage of times when we actually find a good suggestion:",countValid/(len(data)))

Percentage of times when we actually find a good suggestion: 0.599009900990099


In [112]:
print("Average similarity score for the good suggestions:",totalSimilarity/countValid)

Average similarity score for the good suggestions: 0.7961172274321564


In [95]:
print("Average similarity score for the bad suggestions as the closest:",wrong_similarity/countValid)

Average similarity score for the bad suggestions as the closest: 0.5314548606714926


In [ ]:
with open ("eval.json","w") as file:
  json.dump(data, file,indent=4)

In [ ]:
from google.colab import files
files.download("eval.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>